In [1]:
import pandas as pd
import glob

In [2]:
#Данные представленные в таблице имеют лишь колонку с датой и количеством. 
#Необходимо создать новый список который будет содержать код определяющий страну и вид энергии.
#В этом поможет файл "BP_metadata.csv".
#Из него нам потребуется только колонка с кодом и колонка с описанием.


In [3]:
df_meta = pd.read_csv('data/BP_metadata.csv', usecols = ['code', 'description'])
df_meta.head()

,code,description
0,BIOFUEL_PROD_D_ARG,Biofuel Production - Daily Average - Argentina...
1,BIOFUEL_PROD_D_AUS,Biofuel Production - Daily Average - Australia...
2,BIOFUEL_PROD_D_AUT,Biofuel Production - Daily Average - Austria. ...
3,BIOFUEL_PROD_D_BEL,Biofuel Production - Daily Average - Belgium. ...
4,BIOFUEL_PROD_D_BRA,Biofuel Production - Daily Average - Brazil. 1...


In [4]:
#Выведем строки в описании которых присутствует "Russian" и создадим соответствующий список и сохраним в новый файл
meta_rus = df_meta[df_meta['description'].str.contains('Russian')]
meta_rus.head()

,code,description
141,C02_EMMISSIONS_RUS,Carbon Dioxide (CO2) Emmissions - Russian Fede...
234,COAL_CONSUM_O_RUS,Coal Consumption - Oil Equivalent - Russian Fe...
317,COAL_PROD_O_RUS,Coal Production - Oil Equivalent - Russian Fed...
337,COAL_PROD_RUS,Coal Production - Russian Federation. Million ...
425,ELEC_GEN_RUS,Electricity Generation - Russian Federation. T...


In [5]:
meta_rus.to_csv('data/meta/meta_rus.csv', index=False)

In [6]:
#Аналогично сделаем для стран Китай и Германия.
meta_chi = df_meta[df_meta['description'].str.contains('China')]
meta_chi.head()

,code,description
7,BIOFUEL_PROD_D_CHN,Biofuel Production - Daily Average - China. 10...
42,BIOFUEL_PROD_O_CHN,Biofuel Production - Oil Equivalent - China. T...
84,C02_EMMISSIONS_CHN,Carbon Dioxide (CO2) Emmissions - China. Milli...
100,C02_EMMISSIONS_HKG,Carbon Dioxide (CO2) Emmissions - China Hong K...
177,COAL_CONSUM_O_CHN,Coal Consumption - Oil Equivalent - China. Mil...


In [7]:
meta_chi.to_csv('data/meta/meta_chi.csv', index=False)

In [8]:
meta_ger = df_meta[df_meta['description'].str.contains('Germany')]
meta_ger.head()

,code,description
9,BIOFUEL_PROD_D_DEU,Biofuel Production - Daily Average - Germany. ...
44,BIOFUEL_PROD_O_DEU,Biofuel Production - Oil Equivalent - Germany....
88,C02_EMMISSIONS_DEU,Carbon Dioxide (CO2) Emmissions - Germany. Mil...
181,COAL_CONSUM_O_DEU,Coal Consumption - Oil Equivalent - Germany. M...
265,COAL_PROD_DEU,Coal Production - Germany. Million Tonnes. Com...


In [9]:
meta_ger.to_csv('data/meta/meta_ger.csv', index=False)

In [10]:
#Далее нам нужно объеденить таблицы, содержащие информацию по странам. Чтобы по каждой стране был отдельной DataFrame.
# Создадим DataFrame и изменим колонку "Value" на код энергии.

In [11]:
df_C02_EMMISSIONS_RUS = pd.read_csv('data/RUS/BP-C02_EMMISSIONS_RUS.csv').rename(columns={"Date": "Date", "Value": "C02_EMMISSIONS_RUS"})
df_C02_EMMISSIONS_RUS.head()

,Date,C02_EMMISSIONS_RUS
0,2020-12-31,1482.188474
1,2019-12-31,1595.685634
2,2018-12-31,1605.961797
3,2017-12-31,1548.600760
4,2016-12-31,1566.991267


In [12]:
#Этот процесс проделаем в цикле

In [13]:
path = 'data/RUS'
list_name = glob.glob(path + "/*.csv")
for i in range(len(list_name)):
    list_name[i] = list_name[i].replace('data/RUS\\BP-', '')
    list_name[i] = list_name[i].replace('.csv', '')
ls = []
for i in range(len(list_name)):
    ls.append(pd.DataFrame(pd.read_csv('data/RUS/BP-' + list_name[i] + '.csv').rename(columns={"Date": "Date", "Value": list_name[i]})))    


In [14]:
ls[0].head()

,Date,C02_EMMISSIONS_RUS
0,2020-12-31,1482.188474
1,2019-12-31,1595.685634
2,2018-12-31,1605.961797
3,2017-12-31,1548.600760
4,2016-12-31,1566.991267


In [15]:
#Теперь у нас есть список со всеми данными для России. Объеденим их в один DataFrame

In [16]:
df_RUS = pd.DataFrame({"Date": []})
for i in range(len(ls)):
    df_RUS = pd.merge(left=df_RUS, right=ls[i], on='Date', how='outer')
df_RUS.index = df_RUS.Date
del df_RUS['Date']

In [17]:
#Транспонируем для удобства.

In [18]:
df_RUS = df_RUS.transpose()
df_RUS.head()

Date,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31,2012-12-31,2011-12-31,...,1974-12-31,1973-12-31,1972-12-31,1971-12-31,1970-12-31,1969-12-31,1968-12-31,1967-12-31,1966-12-31,1965-12-31
C02_EMMISSIONS_RUS,1482.188474,1595.685634,1605.961797,1548.600760,1566.991267,1549.537429,1579.221849,1581.054684,1605.04989,1591.072886,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COAL_CONSUM_O_RUS,NaN,NaN,88.031641,83.930000,89.250000,92.148000,87.551800,90.517000,98.42560,94.017000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COAL_PROD_O_RUS,NaN,NaN,220.154320,205.800000,193.970000,186.368000,176.591800,173.117000,168.28560,157.577000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COAL_PROD_RUS,399.664238,440.905783,441.576471,412.540341,386.613471,372.482353,357.396992,355.231000,358.30000,337.400000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ELEC_GEN_RUS,1085.418730,1118.143230,1109.197790,1091.184000,1090.970224,1067.543299,1064.196500,1059.085500,1069.28930,1054.857500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#Проделаем то же самое с Китаем и Германией

In [20]:
path = 'data/GER'
list_name = glob.glob(path + "/*.csv")
for i in range(len(list_name)):
    list_name[i] = list_name[i].replace('data/GER\\BP-', '')
    list_name[i] = list_name[i].replace('.csv', '')
ls = []
for i in range(len(list_name)):
    ls.append(pd.DataFrame(pd.read_csv('data/GER/BP-' + list_name[i] + '.csv').rename(columns={"Date": "Date", "Value": list_name[i]})))  

df_GER = pd.DataFrame({"Date": []})
for i in range(len(ls)):
    df_GER = pd.merge(left=df_GER, right=ls[i], on='Date', how='outer')

df_GER.index = df_GER.Date
del df_GER['Date']
df_GER = df_GER.transpose()
df_GER.head()

Date,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31,2012-12-31,...,1974-12-31,1973-12-31,1972-12-31,1971-12-31,1970-12-31,1969-12-31,1968-12-31,1967-12-31,1966-12-31,1965-12-31
BIOFUEL_PROD_D_DEU,54.258232,62.968571,65.753713,62.955768,61.276140,59.869596,59.358525,64.385547,57.611507,53.943981,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BIOFUEL_PROD_O_DEU,NaN,NaN,NaN,3444.890556,3293.476163,3227.645741,3191.265936,3460.226765,3073.430000,2885.661000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C02_EMMISSIONS_DEU,NaN,604.875456,681.482545,734.490511,760.978787,770.511939,755.688173,751.159522,797.630364,773.001264,...,1085.32047,1116.424396,1067.326703,1045.121274,1043.394269,1011.924371,944.755393,892.416120,901.037798,910.413846
COAL_CONSUM_O_DEU,NaN,NaN,NaN,66.399159,71.533128,76.520636,78.680639,79.612640,82.837370,80.487891,...,140.33700,139.736000,139.283000,142.681000,149.097000,150.985424,146.641361,144.566159,150.072178,160.746048
COAL_PROD_DEU,126.000000,107.400000,131.300000,168.842000,175.122000,175.396000,184.288000,185.795000,190.561000,196.202000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
path = 'data/CHI'
list_name = glob.glob(path + "/*.csv")
for i in range(len(list_name)):
    list_name[i] = list_name[i].replace('data/CHI\\BP-', '')
    list_name[i] = list_name[i].replace('.csv', '')
ls = []
for i in range(len(list_name)):
    ls.append(pd.DataFrame(pd.read_csv('data/CHI/BP-' + list_name[i] + '.csv').rename(columns={"Date": "Date", "Value": list_name[i]})))

df_CHI = pd.DataFrame({"Date": []})
for i in range(len(ls)):
    df_CHI = pd.merge(left=df_CHI, right=ls[i], on='Date', how='outer')

df_CHI.index = df_CHI.Date
del df_CHI['Date']
df_CHI = df_CHI.transpose()
df_CHI.head()

Date,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31,2012-12-31,...,1974-12-31,1973-12-31,1972-12-31,1971-12-31,1970-12-31,1969-12-31,1968-12-31,1967-12-31,1966-12-31,1965-12-31
BIOFUEL_PROD_D_CHN,63.899017,57.520908,54.669214,45.152767,46.817161,38.542840,40.185640,47.190628,42.487881,38.067688,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BIOFUEL_PROD_O_CHN,NaN,NaN,NaN,3099.378410,2146.587912,1811.374580,2038.898809,2608.550357,2346.005021,2110.356630,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C02_EMMISSIONS_CHN,NaN,9899.334721,9810.456419,9652.687227,9466.360373,9278.978313,9279.731356,9293.185855,9247.425491,9004.244462,...,1006.768535,987.002301,945.376724,881.701156,748.50713,582.4737,476.68279,475.920674,530.349045,488.525419
COAL_CONSUM_O_CHN,NaN,NaN,NaN,1906.725000,1890.425612,1889.093134,1913.958685,1954.484357,1969.073011,1927.793093,...,202.800000,204.965000,202.315000,191.305000,165.92000,130.4050,107.99000,108.800000,122.400000,114.420000
COAL_PROD_CHN,4126.000000,3901.577100,3846.332000,3697.735800,3523.561800,3410.604000,3746.541600,3873.919000,3974.322000,3945.128100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
#Сохраним результаты. На этом первая часть подготовки данных для анализа заканчивается

In [24]:
df_RUS.to_csv('data/results/RUS.csv')
df_GER.to_csv('data/results/GER.csv')
df_CHI.to_csv('data/results/CHI.csv')